# Stroke Prediction Model

Final stroke prediction model based on the research in the Stroke_Prediction_Model_Searching_2 on the healthcare-dataset-stroke-data dataset.

Source: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset

In [18]:
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.dummy import DummyClassifier

from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imPipeline

In [2]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    ''' Simple class for selecting given attributes form given data frame. 
        Inspired by https://sklearn-features.readthedocs.io/en/latest/_modules/transformers.html '''
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

class MyLabelBinarizer(LabelBinarizer):
    ''' Customized LabelBinarizer to make it work with pipeline which require two arguments.
        Returs the same as LabelBinarizer. '''
    def fit_transform(self, X, y=None):
        return super(LabelBinarizer, self).fit_transform(X)

class CustomLimitedImputer(BaseEstimator, TransformerMixin):
    ''' Simple customized imputer to change the following:
        smoking_status to "never smoked" if age < 10 and smoking_status = "Unknown"
        work_type to "children" if age < 17 and swork_type = "Never_worked" '''
    def __init__(self, attribute_names):
        assert all(attr in ['smoking_status', 'work_type'] for attr in attribute_names), 'Only smoking_status and work_type are supported'
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.copy()
        for attr in self.attribute_names:
            if attr == 'smoking_status':
                X.loc[(X.age < 10) & (X.smoking_status == 'Unknown'), 'smoking_status'] = 'never smoked'
            elif attr == 'work_type':
                X.loc[(X.age < 17) & (X.work_type == 'Never_worked'), 'work_type'] = 'children'
        X.drop(['age'], axis=1, inplace=True)
        return X.values

# pipelines
cat_yn_pipeline = Pipeline([
        ("select_bin", DataFrameSelector(['ever_married'])),
        ("bin_encoder", MyLabelBinarizer()),
    ])

cat_oh_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(['smoking_status', 'work_type', 'age'])), # age is used as a CustomLimitedImputer parameter
        ("imputer", CustomLimitedImputer(['smoking_status', 'work_type'])),
        ("cat_encoder", OneHotEncoder(sparse=False)),
    ])

num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(['age', 'avg_glucose_level'])),
        ("scale", StandardScaler()),
    ])

# final preprocessing pipeline
preprocess_pipeline = FeatureUnion(transformer_list=[
        ('cat_yn_pipeline', cat_yn_pipeline),
        ('cat_oh_pipeline', cat_oh_pipeline),
        ('num_pipeline', num_pipeline),
    ])

In [3]:
data = pd.read_csv('Data/healthcare-dataset-stroke-data.csv', index_col='id')

In [4]:
data

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0


Split data into three datasets: train, validation and test.

In [5]:
X_train_val, X_test, y_train_val, y_test = train_test_split(data[data.columns[:-1]], data.stroke, test_size=0.2, random_state=24, stratify=data.stroke)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=24, stratify=y_train_val)

In [7]:
X_train = preprocess_pipeline.fit_transform(X_train)
X_val = preprocess_pipeline.transform(X_val)
X_test = preprocess_pipeline.transform(X_test)

Quick look on the data.

In [9]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)
print(y_train.value_counts())
print(y_val.value_counts())

(3270, 12)
(818, 12)
(1022, 12)
(3270,)
(818,)
(1022,)
0    3111
1     159
Name: stroke, dtype: int64
0    778
1     40
Name: stroke, dtype: int64


Data are imbalanced. There are many more no-stoke individuals than stroke ones so let's perform the resampling. More details can be found in "Stroke_Prediction_Model_Searching_v2" notebook.

In [8]:
y_train.value_counts()

0    3111
1     159
Name: stroke, dtype: int64

Pipeline for oversampling and undersampling. The values and algorithms have been chosen based on the research covered in the "Stroke_Prediction_Model_Searching_v2" notebook.

In [10]:
oversample_v = 0.5
undersample_v = 0.7
oversample_tq = ADASYN(sampling_strategy=oversample_v, random_state=24)
undersample_tq = RandomUnderSampler(sampling_strategy=undersample_v, random_state=24)
resample_pipeline = imPipeline([('oversampling', oversample_tq), ('undersampling', undersample_tq)])
X_train_ou, y_train_ou = resample_pipeline.fit_resample(X_train, y_train)

In [11]:
y_train_ou.value_counts()

0    2195
1    1537
Name: stroke, dtype: int64

Before model training, the metric needs to be selected.
The choice is F2 score because is better for the models where positive class and preventing false negatives are more important.

In [12]:
f2_scorer = make_scorer(fbeta_score, beta=2)

Now let's create benchmark for score using DummyClassifier.

In [13]:
# only 1
dc = DummyClassifier(strategy='constant', constant=1, random_state=24) 

# randomly sampled but in stratified manner
# dc = DummyClassifier(strategy='c', random_state=24)

dc.fit(X_train, y_train)
dc_y_pred = dc.predict(X_train)

In [14]:
fbeta_score(y_train, dc_y_pred, beta=2)

0.2035330261136713

In case the model predicts only 1 the f2_score is 0.2035.
<br>
In case the model predicts randomly in balanced way the f2_score is 0.0624.

To make sure the datasets are split equally (in terms of classes) during cross validation let's use the Stratified Fold. Furthermore the splits will be produced differently in a couple repetitions.

In [15]:
splitter = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

Function for evaluating the cross validation of train dataset and validation dataset prediction.

In [16]:
def cross_val_scores(model, X, y, cv):
    ''' Calculate the cross validation scores based on f2_scores metric and then the mean of them.
        Parameters:
            model    - model to be used for fitting data and predictions
            X        - X train dataset
            y        - y train dataset
            cv       - Cross validation splitting strategy
        Returns:
            The mean value of calculated f2 scores. '''
    scores = cross_val_score(model, X, y, cv=cv, scoring=f2_scorer)
    return scores.mean()

def get_val_pred_score(model, X, y, X_v, y_v):
    ''' Calculate the f2_score of prediction for given model. 
        Parameters:
            model    - model to be used for fitting data and predictions
            X        - X train dataset
            y        - y train dataset
            X_v      - X validation dataset (works for test dataset as well)
            y_v      - y validation dataset (works for test dataset as well)
        Result:
            f2_score of predicted values. '''
    model.fit(X, y)
    y_pred = model.predict(X_v)
    return fbeta_score(y_v, y_pred, beta=2)

#### Now let's create the models. The two best performing based on the "Stroke_Prediction_Model_Searching_v2" notebook, are:
SGDClassifier      - the best one from linear clissifiers
AdaBoostClassifier - the best one from tree based classifiers

Val_score   Algorithm(hyperparameters)
0.4381      SGDClassifier(alpha=0.0004, penalty='elasticnet', l1_ratio=0.05, random_state=24)

0.4103      AdaBoostClassifier(base_estimator=ExtraTreeClassifier(max_depth=2, min_samples_leaf=10, random_state=24),
                             learning_rate=0.05, n_estimators=900, random_state=24)

Training the SGDClassifier model. More details in "Stroke_Prediction_Model_Searching_v2" notebook.

In [19]:
sgd_clf = SGDClassifier(alpha=0.0004, penalty='elasticnet', l1_ratio=0.05, random_state=24)

Check the cross validated results.

In [21]:
cross_val_scores(sgd_clf, X_train_ou, y_train_ou, splitter)

0.773184552660715

Check the results for validation data set.

In [22]:
get_val_pred_score(sgd_clf, X_train_ou, y_train_ou, X_val, y_val)

0.43814432989690727

Verify the model on the previously separated test data which have not been seen and not used for evaluating by the model previously.

In [23]:
sgd_clf.fit(X_train_ou, y_train_ou)

SGDClassifier(alpha=0.0004, l1_ratio=0.05, penalty='elasticnet',
              random_state=24)

In [24]:
y_pred = sgd_clf.predict(X_test)

In [25]:
fbeta_score(y_test, y_pred, beta=2)

0.34934497816593885

The confusion matrix for test dataset.

In [26]:
confusion_matrix(y_test, y_pred)

array([[746, 226],
       [ 18,  32]], dtype=int64)

#### The results are:
    380% and 1239% better than the benchmark* for train dataset
    215% and 702% better than the benchmark* for verification dataset
    172% and 560% better than the benchmark* for test dataset

#### For the test dataset which was unseen during the whole research:
    The 64% (32 of 50) strokes have been predicted correctly. 
    The 23% (226 of 972) non-strokes where considered wrongly as strokes. 
    
________________________________
**benchmark for train dataset 0.2035 (only 1) and 0.0624 (randomly balanced)*

Training the AdaBoostClassifier model. More details in "Stroke_Prediction_Model_Searching_v2" notebook.

In [27]:
ab_clf = AdaBoostClassifier(base_estimator=ExtraTreeClassifier(max_depth=2, min_samples_leaf=10, random_state=24),
                             learning_rate=0.05, n_estimators=900, random_state=24)

Check the cross validated results.

In [28]:
cross_val_scores(ab_clf, X_train_ou, y_train_ou, splitter)

0.8111404412281389

Check the results for validation data set.

In [29]:
get_val_pred_score(ab_clf, X_train_ou, y_train_ou, X_val, y_val)

0.41033434650455936

Verify the model on the previously separated test data which have not been seen and not used for evaluating by the model previously.

In [30]:
ab_clf.fit(X_train_ou, y_train_ou)

AdaBoostClassifier(base_estimator=ExtraTreeClassifier(max_depth=2,
                                                      min_samples_leaf=10,
                                                      random_state=24),
                   learning_rate=0.05, n_estimators=900, random_state=24)

In [33]:
y_pred = ab_clf.predict(X_test)

In [34]:
fbeta_score(y_test, y_pred, beta=2)

0.35175879396984927

The confusion matrix for test dataset.

In [35]:
confusion_matrix(y_test, y_pred)

array([[802, 170],
       [ 22,  28]], dtype=int64)

### The results are:
    399% and 1300% better than the benchmark* for train dataset
    202% and 658% better than the benchmark* for verification dataset++
    173% and 564% better than the benchmark* for test dataset

#### For the test dataset which was unseen during the whole research:
    The 56% (28 of 50) strokes have been predicted correctly. 
    The 17% (226 of 972) non-strokes where considered wrongly as strokes.

________________________________
**benchmark for train dataset 0.2035 (only 1) and 0.0624 (randomly balanced)*

### Steps description:
    1. In the Stroke_Prediction_Data_Exploration notebook the following were performed:
            - Data overview
            - Features and samples exploratory analysis
            - Data evaluation
    
    2. In the Stroke_Prediction_Cleansing_and_Preprocessing notebook the pipelines were build to perform all actions based            on findings from the previous steps (notebook).
    
    3. In the Stroke_Prediction_Model_Searching and Stroke_Prediction_Model_Searching_2 notebooks all research can be found.
       Checked classification algorithms:
           - LogisticRegression
           - LinearDiscriminantAnalysis
           - GaussianNB
           - KNeighborsClassifier
           - SVC
           - LinearSVC
           - SGDClassifier
           - DecisionTreeClassifier
           - RandomForestClassifier
           - ExtraTreesClassifier
           - BaggingClassifier
           - GradientBoostingClassifier
           - AdaBoostClassifier
           - HistGradientBoostingClassifier
       Checked outlier/anomaly detection algorithms:
           - OneClassSVM
           - IsolationForest
           - LocalOutlierFactor
       Checked resampling:
           - SMOTE
           - BorderlineSMOTE
           - SVMSMOTE
           - SMOTENC
           - ADASYN
           - RandomUnderSampler
           - CondensedNearestNeighbour
           - TomekLinks
           - EditedNearestNeighbours
           - NeighbourhoodCleaningRule
       Checked metrics:
           - recall
           - f2-score
       Hyperparameters tuning techniques:
           - RandomizedSearchCV
           - GridSearchCV
       

### Summary:
    From all above the best performing were:
      Models:
        - SGDClassifier      - the best one from linear clissifiers
        - AdaBoostClassifier - the best one from tree based classifiers
        
      Sampling techniques:
        - ADASYN             - for oversampling
        - RandomUnderSampler - for undersampling
        
    Used metric was f2-score which favoritize positive class and preventing false negatives to appear.
    
    Achieved f2-scores for unseen test dataset are:
        - SGDClassifier:      0.3493
        - AdaBoostClassifier: 0.3518
    
    Despite that the benchmark (DummyClassifier) was beaten in both cases, the results are far away from the desired ones.
    Models have pretty high numbers of False Negative and False Positive. 
    It is out of the question to put such models as a replacement of medical supervision.

#### Ideas for improvement:
    1. Collect more samples especially for stroke class. Could be the problem cannot be solved due to lack of the positive  samples.
    2. Reduce the feature number to simplify the classification decision making.
    3. Research neural networks models.
    4. Do the research on stroke medical papers to understand problem better.
    5. Ask better questions during features and samples exploratory analysis.
    6. Research more algorithms combinations.

#### Lessons learned:
    1. Small imbalanced datasets may have strong tendency for overfitting. 
    2. Combining oversampling with undersampling can be beneficial. 
    3. Trees based algorithms are highly adaptive what can lead to overfitting.
    4. Asking "what can go wrong" is a good approach in the initial phase. 
    5. Cross validation does not replace the validation dataset.

#### Final words:
    The results are not on the desired level. That kind of models cannot be use as a replacement of medical expertise.
    There is still some potential in the research but let's go further to work on other projects and come to that problem in future smarter and with more experience. 